In [ ]:
"""
Notebook utilizado para apresentar uma breve introdução ao processamento de sinais.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.

Protegido por licença GPL v3.0. O uso e a reprodução são livres para fins educacionais
desde que citada a fonte.

Prof. Dr. Emerson Rodrigo Almeida
Universidade Federal de Uberlândia
Instituto de Geografia - Curso de Geologia
emerson.almeida@ufu.br

Última modificação: 05/02/2021
"""

# Instruções iniciais

Neste notebook você irá aprender como a digitalização de um sinal analógico é feita corretamente a partir dos intervalos de amostragem. Para isto leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo contém as funções utilizadas nos cálculos e as funções utilizadas para gerar as figuras. Execute-a uma vez antes de executar qualquer outra parte do código e **não altere nada no código apresentado na célula** para não criar problemas na execução do código.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


# funções para os cálculos

def gera_funcao_ref(a, f, t_max):
    """
    Gera uma função obtida a partir de uma sobreposição de funções cosseno, onde cada uma delas é caracterizada por
    uma amplitude A e frequência f na forma s(t) = A * cos(2 * pi * f * t). A sobreposição destas funções será o sinal
    analógico de referência. Esta função pode ser amostrada em qualquer instante de tempo que se queira, de forma 
    que esta é a melhor forma de representar um sinal contínuo para os objetivos desta demonstração.
    """

    dt = t_max/1000
    tt = np.arange(-t_max, (2*t_max) + dt, dt)                        # eixo de tempo estendido
    
    s = np.zeros(np.shape(tt))
    
    for i in range(len(a)):
        s = s + (a[i] * np.cos(2 * np.pi * f[i] * tt))
        
    return interp1d(tt, s)      # função que caracteriza o sinal analógico
    

def gera_sinal_analogico(t_max, f_sinal):
    """
    Gera a representação do sinal analógico. O que faz na verdade é calcular as amplitudes da função cosseno com inervalo
    de amostragem curto o suficiente para que ela possa ser visualizada como um sinal analógico contínuo.
    """
    
    dt = t_max/1000
    tt = np.arange(-t_max, (2*t_max), dt)

    return f_sinal(tt), tt
    
    
def amostra_sinal(t_max, dt, f_sinal):
    """
    Faz a amostragem do sinal calculando as amplitudes da função cosseno de acordo com o intervalo
    de amostragem definido.
    """
    
    t_amostrado = np.arange(-t_max, t_max*2, dt)
    sinal_amostrado = f_sinal(t_amostrado)      # representação aproximada do sinal "analógico"
    
    return sinal_amostrado, t_amostrado


def recupera_sinal(t_in, s_in):
    """
    Interpola as amplitudes que foram amostradas da função cosseno para demonstrar como seria o
    comportamento real do sinal recuperado a partir destas amostras.
    """
    
    s_out = interp1d(t_in, s_in, kind='cubic')
    
    t_max = np.max(t_in)
    dt = t_max/1000
    tt = np.arange(-t_max, (2*t_max) + dt, dt)
    i1 = np.abs(tt - t_in[0]).argmin()
    i2 = np.abs(tt - t_in[-1]).argmin()

    return s_out(tt[i1+1:i2]), tt[i1+1:i2]
    

# funções de figuras

def plota_amostragem(tt1, s1, tt2, s2, t_w, titulo):
    """
    Plota as amplitudes amostradas em relação ao sinal analógico original
    """
        
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(tt1, s1, '-r', label='Sinal original')
    ax.plot(tt2, s2, '.b', label='Amplitudes amostradas')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title(titulo, fontsize=14)
    ax.set_xlim([0, t_w])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)

    
def plota_sinal(tt, a, t_w, titulo):
    """
    Plota a função cosseno calculada a intervalos de tempo pequenos o suficiente para que se possa 
    fazer uma representação do sinal analógico original.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(tt, a)
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title(titulo, fontsize=14)
    ax.set_xlim([0, t_w])
    ax.grid()
    
    
def plota_representacao(tt_am, s_am, tt_rec, s_rec, t_w, titulo):
    """
    Plota a interpolação do sinal feita a partir das amostras obtidas do sinal analógico.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(tt_rec, s_rec, '--b', label='Sinal recuperado', linewidth=1)
    ax.plot(tt_am, s_am, '.b', label='Amplitudes amostradas')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title(titulo, fontsize=14)
    ax.set_xlim([0, t_w])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)
    
    
def plota_comparacao(tt1, s1, tt2, s2, t_w, titulo):
    """
    Plota a interpolação do sinal feita a partir das amostras obtidas do sinal analógico e a sobrepõe à 
    representação do sinal analógico original.
    """
    
    fig, ax = plt.subplots(figsize=(15,3))
    ax.plot(tt1, s1, '-r', label='Sinal original')
    ax.plot(tt2, s2, '--b', label='Sinal recuperado')
    ax.set_xlabel("Tempo (s)", fontsize=14)
    ax.set_ylabel("Amplitude (ua)", fontsize=14)
    ax.set_title(titulo, fontsize=14)
    ax.set_xlim([0, t_w])
    ax.grid()
    ax.legend(loc='lower right', fontsize=12)

### Passo 2 - Características do sinal analógico

Aqui iremos configurar as características que queremos no nosso sinal analógico. Nosso sinal analógico será caracterizado por uma sobreposição de cinco funções cosseno, cada uma possuindo suas próprias características de amplitude ($A_1, A_2, ..., A_5$), dada em unidades de amplitude (ua) e de frequência ($f_1, f_2, ..., f_5$), dada em Hertz (Hz). Assim, teremos:

$$ s_1(t) = A_1 cos(2 \pi f_1 t)$$
$$ s_2(t) = A_2 cos(2 \pi f_2 t)$$
$$ s_3(t) = A_3 cos(2 \pi f_3 t)$$
$$ s_4(t) = A_4 cos(2 \pi f_4 t)$$
$$ s_5(t) = A_5 cos(2 \pi f_5 t)$$

$$ y(t) = s_1(t) + s_2(t) + s_3(t) + s_4(t) + s_5(t)$$

Cada função cosseno $s_n(t)$ apresenta uma *componente* de sinal que constitui o sinal $y(t)$, e todas as componentes são definidas dentro de uma mesma janela temporal de 5.0 s. As componentes de sinal que iremos utilizar têm amplitudes $A_1$ = 1.0 ua, $A_2$ = 0.5 ua, $A_3$ = 1.5 ua, $A_4$ = 2.75 ua e $A_5$ = 0.8 ua e frequências $f_1$ = 1.0 Hz, $f_2$ = 3.0 Hz, $f_3$ = 15.0 Hz, $f_4$ = 0.50 Hz e $f_5$ = 7.0 Hz. Execute a célula abaixo para estabelecer as configurações dos parâmetros do sinal analógico. Não altere nada por enquanto.

In [ ]:
a1 = 1.00      # amplitude da componente s_1(t) em unidades de amplitude
f1 = 1.00      # frequência da componente s_1(t) em Hertz

a2 = 0.50      # amplitude da componente s_2(t) em unidades de amplitude
f2 = 3.00      # frequência da componente s_2(t) em Hertz

a3 = 1.50      # amplitude da componente s_3(t) em unidades de amplitude
f3 = 15.0      # frequência da componente s_3(t) em Hertz

a4 = 2.75      # amplitude da componente s_4(t) em unidades de amplitude
f4 = 0.50      # frequência da componente s_4(t) em Hertz

a5 = 0.80      # amplitude da componente s_5(t) em unidades de amplitude
f5 = 7.00      # frequência da componente s_5(t) em Hertz


janela_tempo = 5.0              # janela temporal em segundos

Agora vamos gerar o sinal analógico propriamente dito. Entretanto, temos uma limitação aqui. O computador trabalha apenas com informações discretas, i.e., é preciso ter valores específicos de tempo e amplitude para plotar corretamente a figura, e isto por si só já caracteriza uma espécie de digitalização. Iremos contornar esta limitação criando nosso sinal analógico a partir de uma função cosseno definida matematicamente. Isto permite que possamos obter valores de amplitude em qualquer instante de tempo que queiramos, já que a função cosseno é uma função contínua. 

Execute a célula abaixo para obter a expressão matemática que representará nosso sinal analógico.

In [ ]:
funcao_sinal = gera_funcao_ref([a1, a2, a3, a4, a5], [f1, f2, f3, f4, f5], janela_tempo)

Está achando tudo muito abstrato até aqui? Não se desespere, pois agora vamos gerar uma representação visual do sinal analógico. O que fazemos aqui é avaliar a nossa função cosseno em pontos que estão tão próximos entre si que podem ser interpolados sem prejuízo para a representação da sua continuidade.

Execute a célula abaixo para visualizar o sinal analógico.

In [ ]:
sinal_analogico, t_analogico = gera_sinal_analogico(janela_tempo, funcao_sinal)
plota_sinal(t_analogico, sinal_analogico, janela_tempo, "Figura 1. Representação do sinal analógico original")

**Perguntas:** Você consegue identificar visualmente todas as componentes neste sinal? Dica: se você quiser visualizar uma única componente do sinal basta configurar a amplitude de cada uma das outras quatro componentes para zero na primeira célula deste passo. Por exemplo, para visualizar apenas a componente $s_3(t)$, configure as amplitudes $A_1$, $A_2$, $A_4$ e $A_5$ para 0.0. Não se esqueça de restabelecer os valores padrão de amplitude de cada componente antes de continuar a demonstração.

### Passo 03 - Amostragem

A amostragem do sinal com múltiplas componentes obedece aos mesmos critérios estabelecidos pela Teoria da Amostragem que vimos anteriormente. Porém, antes tínhamos apenas uma componente e agora temos várias. Qual delas deverá ser escolhida? A resposta para esta pergunta é: a frequência mais alta que você quer registrar. Tome como exemplo a componente $s_3(t)$. Ela tem frequência de 15 Hz, que é a mais alta de todas e a única acima de 10 Hz. Se for do seu interesse incluir esta frequência na amostragem você deve considerá-la no processo de amostragem. Por outro lado, se você considerar que esta frequência esteja associada a uma componente de ruído e não interessa ao registro do seu sinal, deverá considerar como frequência máxima a frequência do sinal $s_5(t)$, que é de 7.0 Hz. 

A frequência máxima capaz de ser amostrada por um determinado intervalo de amostragem é denominada **frequência de Nyquist** ($f_N$). O intervalo de amostragem necessário para registrar esta frequência pode ser obtido da mesma forma que fizemos anteriormente, através do critério estabelecido pelo Teorema da Amostragem:

$$\Delta t_{am_{N}} = \frac{1}{f_{am_{N}}} = \frac{1}{2 * f_N}$$

Vamos então definir esta frequência de amostragem para nosso sinal executando a célula abaixo:

In [ ]:
f_N = 15.0          # frequência de Nyquist, em Hz
f_am = 2 * f_N      # frequência de amostragem

e, em seguida, executando a célula abaixo para visualizar as amplitudes que foram amostradas (pontos azuis) no nosso sinal analógico (linha vermelha):

In [ ]:
sinal_amostrado, t_amostrado = amostra_sinal(janela_tempo, 1/f_am, funcao_sinal)
plota_amostragem(t_analogico, sinal_analogico, t_amostrado, sinal_amostrado, janela_tempo, \
                 "Figura 2. Sinal amostrado a uma frequência de " + str(f_am) + " amostras por segundo")

**Pergunta:** Qual o intervalo de amostragem utilizado?

### Passo 04 - Recuperação do sinal

Quando o sinal é amostrado corretamente é possível recuperá-lo através de sistemas dedicados, que fogem ao escopo desta demonstração. Aqui exemplificaremos a recuperação do sinal através de uma simples interpolação.

Quando registramos as amostras de um sinal, as informações sobre a amplitude do sinal e sobre os intantes de tempo em que estas amplitudes foram medidas são tudo o que temos para trabalhar. Não há informação alguma sobre o comportamento do sinal, que pode ser uma função cosseno, uma função seno, uma sobreposição de ambas, ou qualquer outra coisa. Desta forma, vamos pegar as amostras que visualizamos na Figura 2 e interpolar um novo sinal a partir delas executando a célula abaixo:

In [ ]:
sinal_rec, t_rec = recupera_sinal(t_amostrado, sinal_amostrado)
plota_representacao(t_amostrado, sinal_amostrado, t_rec, sinal_rec, janela_tempo, "Figura 3. Sinal recuperado da amostragem")

Assim como um software de análise de dados geofísicos não tem ideia sobre o comportamento do sinal verdadeiro, não há qulquer informação pré-determinada sobre as características do sinal original na interpolação realizada nesta demonstração. Perceba que, ainda assim, é possível recuperar um sinal com as características de uma função cosseno. Mas será que o sinal recuperado é fiel ao sinal original?

Execute a célula abaixo para visualizar uma compraração entre o sinal recuperado (em azul) e o sinal original (em vermelho).

In [ ]:
plota_comparacao(t_analogico, sinal_analogico, t_rec, sinal_rec, janela_tempo, \
                 "Figura 4. Comparação entre o sinal original e o sinal recuperado da amostragem")

**Pergunta:** Você considera que a recuperação foi bem feita? Qual a frequência máxima do sinal original? Qual a frequência máxima que você consegue identificar no sinal recuperado?

# Exercícios

**1 -** Volte à célula em que você definiu as características das componentes do sinal analógico, no início do Passo 02. Modifique os parâmetros de amplitude e frequência das componentes do sinal e verifique qual é o intervalo de amostragem ideal para representar o sinal de forma integral. Para isto, determine o intervalo de amostragem ideal para cada componente e defina qual deles é o mais indicado. Defina os intervalos de amostragem para sinais de 2 Hz, 0.5 Hz, 14 kHz. Ajuste a janela temporal conforme necessário para visualizar corretamente o sinal.

**2 -** Volte agora à primeira linha da primeira célula do Passo 03, onde a frequência de Nyquist foi definida como 15.0 Hz, e modifique este valor para 7.0 Hz. O que aconteceu com a frequência de 15 Hz no sinal recuperado? O que acontece com o sinal recuperado se a frequência de Nyquist for definida como 3.0 Hz? E se for definida como 1.0 Hz? E se for definida como 0.5 Hz?

**3 -** Agora defina a frequencia de Nyquist novamente como 7.0 Hz na primeira célula do Passo 03 e modifique a frequência de amostragem, alterando o número correspondente ao fator que multiplica a frequência de Nyquist para $f_{am} = 5 * f_{N}$, na segunda linha desta célula. Que características pode-se observar no sinal recuperado?

**Observações: a)** Modifique apenas os números. Não modifique os nomes das variáveis, ou o código poderá haver problemas na execução do código. **b)** Utilize números com ponto decimal. **c)** Após modificar os parâmetros do sinal analógico na célula correspondente será necessário executá-la e executar as células seguintes novamente, uma a uma e em ordem, para que as figuras sejam geradas corretamente. **d)** Caso tenha qualquer problema com a execução do código experimente clicar em "Kernel > Restart & Clear Output" na barra acima. Se o problema persistir entre em contato comigo reportando o problema em detalhes.